In [1]:
import json
import time
import azureml
import logging
from azureml.core.model import Model
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace, Run, Datastore, Experiment
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.train.hyperdrive import *
from azureml.train.dnn import PyTorch
from azureml.widgets import RunDetails

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.0.2


In [2]:
# use this code to set up config file
#subscription_id ='<SUB_ID>'
#resource_group ='<RESOURCE_GROUP>'
#workspace_name = '<WORKSPACE>'

#try:
#   ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#   ws.write_config()
#   print('Workspace configuration succeeded. You are all set!')
#except:
#   print('Workspace not found. TOO MANY ISSUES!!!')
ws = Workspace.from_config()

Found the config file in: C:\projects\pytorchintro\aml_config\config.json


In [3]:
cluster = 'racer'
try:
    compute = ComputeTarget(workspace=ws, name=cluster)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', min_nodes=1, max_nodes=6)
    compute = ComputeTarget.create(ws, cluster, compute_config)
    compute.wait_for_completion(show_output=True)

Found existing compute target


In [4]:
# Create and run experiment
mnist = Experiment(ws, 'pytorchmnist')

# script parameters
script_params={
    '--epochs': 10,
    '--batch': 100,
    '--lr': .001,
    '--model': 'cnn'
}

# Create Estimator
estimator = PyTorch(source_directory='.',
                       compute_target=compute, 
                       entry_script='train.py',
                       script_params=script_params,
                       use_gpu=True)

run = mnist.submit(estimator)

In [5]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
pytorchmnist,pytorchmnist_1545268585383,azureml.scriptrun,Queued,Link to Azure Portal,Link to Documentation


In [6]:
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'NOTSET',…

In [7]:
run.get_file_names()

['azureml-logs/60_control_log.txt',
 'azureml-logs/80_driver_log.txt',
 'outputs/model.pth',
 'outputs/model.onnx',
 'driver_log',
 'azureml-logs/azureml.log',
 'azureml-logs/55_batchai_execution.txt']

In [13]:
model_file = 'outputs/model.pth'
run.download_file(name=model_file, output_file_path='model.pth')
model = Model.register(ws, model_name='PyTorchMNIST', model_path='model.pth', 
                       description='CNN PyTorch Model')

Registering model PyTorchMNIST


In [16]:
myenv = CondaDependencies()
myenv.add_pip_package('numpy')
myenv.add_pip_package('torch')
with open('pytorchmnist.yml','w') as f:
    print('Writing out {}'.format('pytorchmnist.yml'))
    f.write(myenv.serialize_to_string())
    print('Done!')

Writing out pytorchmnist.yml
Done!


In [18]:
from azureml.core.image import ContainerImage, Image

# create image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                runtime="python", 
                                conda_file="pytorchmnist.yml")

image = Image.create(ws, 'pytorchmnist', [model], image_config)
image.wait_for_creation(show_output=True)

Creating image
Running............................................
SucceededImage creation operation finished for image pytorchmnist:1, operation "Succeeded"


In [21]:
from azureml.core.webservice import Webservice, AciWebservice
# create service
aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                            memory_gb=1, 
                                            description='simple MNIST digit detection')
service = Webservice.deploy_from_image(workspace=ws, 
                                    image=image, 
                                    name='pytorchmnist-svc', 
                                    deployment_config=aciconfig)
service.wait_for_deployment(show_output=True)

WebserviceException: Error, there is already a service with name pytorchmnist-svc found in workspace wrecker

In [20]:
with open('deploy.log','w') as f:
    f.write(service.get_logs())

In [ ]:
import json
import requests
service = 'http://13.68.203.69:80/score'
image_url = 'https://thumbs.dreamstime.com/z/red-yellow-white-dishes-cups-shelves-store-dishes-display-99263287.jpg'
r = requests.post(service, json={'image': image_url, 'confidence': .2})   
response = json.loads(r.json())